# Includes

In [ ]:
!pip install git+https://github.com/qubvel/classification_models.git
!pip install --upgrade git+https://github.com/divamgupta/image-segmentation-keras
!pip install imgaug==0.4.0

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-v2nhmgo0
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-v2nhmgo0
  Running command git submodule update --init --recursive -q
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-cp37-none-any.whl size=20030 sha256=696d85e1e7cc53721611415ca5d45efb330aac5c1384ec3ee760e311f498dd71
  Stored in directory: /tmp/pip-ephem-wheel-cache-fw_jjgcb/wheels/de/2b/fd/29a6d33edb8c28bc7d94e95ea1d39c9a218ac500a3cfb1b197
Successfully built image-classifiers
  Cloning https://github.com/divamgupta/image-segmentation-keras to /tmp/pip-req-build-t4k30p3f
  Running command git clone -q https://github.com/divamgupta/image-segmentation-keras /tmp/pip-req-build-t4k30p3f
     |████████████████████████████████| 3.3MB 5.9MB/s 
  Created wheel for keras-segmentation: filename=keras_segmentation-0.3.0-cp37-none-any.whl size=34545 sha256=695c4d4ca6120de82dfa175b

In [ ]:
import cv2
import numpy as np
import os
import itertools
import six
import random
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from glob import glob
import tensorflow as tf
import keras
from classification_models.keras import Classifiers
from keras_segmentation.data_utils.data_loader import image_segmentation_generator
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Prepare dataset

In [ ]:
img_path = '/content/drive/MyDrive/ResNet/data/img/'
proc_img_path = '/content/drive/MyDrive/ResNet/data/proc_img/'
mask_path = '/content/drive/MyDrive/ResNet/data/masks_machine/'
proc_mask_path = '/content/drive/MyDrive/ResNet/data/proc_masks_machine/'

In [ ]:
dirs = os.listdir(img_path)
img_dataset = []
for item in dirs:
        if os.path.isfile(img_path+item):
            im = cv2.imread(img_path+item)
            f, e = os.path.splitext(img_path+item)
            im = cv2.resize(im, (224,224))
            im = im/255
            #cv2.imwrite(proc_img_path + item, im)
            img_dataset.append(im)

In [ ]:
dirs = os.listdir(mask_path)

for item in dirs:
        if os.path.isfile(mask_path+item):
            im = cv2.imread(mask_path+item)
            im[im == 3] = 0
            cv2.imwrite(mask_path + item, im)

In [ ]:
dirs = os.listdir(proc_mask_path)
mask_dataset = []

for item in dirs:
        if os.path.isfile(mask_path+item):
            im = cv2.imread(mask_path+item)
            im = cv2.resize(im, (64,64))
            im[im == 3] = 0
            #cv2.imwrite(proc_mask_path + item, im)
            mask_dataset.append(im)

#Prepare model

In [ ]:
ResNet18, preprocess_input = Classifiers.get('resnet18')

n_classes = 3

# build model
base_model = ResNet18(input_shape=(640,480,3), weights='imagenet', include_top=False)
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
base_model.layers.pop(0)

In [ ]:
base_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               [(None, 640, 480, 3) 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 640, 480, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_39 (ZeroPadding2 (None, 646, 486, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 320, 240, 64) 9408        zero_padding2d_39[0][0]          
____________________________________________________________________________________________

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
train_images = "/content/drive/MyDrive/ResNet/data/img/"
train_annotations = "/content/drive/MyDrive/ResNet/data/masks_machine/"
batch_size = 32
n_classes = 3
input_height = 224
input_width = 224
output_height = 64
output_width = 64
do_augment = True
augmentation_name="aug_all"
custom_augmentation=None
other_inputs_paths=None
preprocessing=None
read_image_type=1

#Training

In [ ]:
train_gen = image_segmentation_generator(
        train_images, train_annotations,  batch_size,  n_classes,
        input_height, input_width, output_height, output_width,
        do_augment=do_augment, augmentation_name=augmentation_name,
        custom_augmentation=custom_augmentation, other_inputs_paths=other_inputs_paths,
        preprocessing=preprocessing, read_image_type=read_image_type)

In [ ]:
for i in train_gen:
  print(i[1].shape)

(32, 4096, 3)
(32, 4096, 3)


KeyboardInterrupt: ignored

In [ ]:
checkpoint = [keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/ResNet/checkpoints/frozen_conv2d.hdf5')]

In [ ]:
history = model.fit(train_gen,
                  steps_per_epoch=256,
                  epochs=5, 
                  callbacks=checkpoint,
                  use_multiprocessing=False, 
                  initial_epoch=0)

Epoch 1/5


ValueError: ignored

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=1e-5),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 224, 224, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_21 (ZeroPadding2 (None, 230, 230, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 112, 112, 64) 9408        zero_padding2d_21[0][0]          
____________________________________________________________________________________________

In [ ]:
for i in range(65,106):
  model.get_layer(index=i).trainable = True

In [ ]:
checkpoint_fine = [keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/ResNet/output/fine.hdf5')]

In [ ]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=256,
                              epochs=5,
                              callbacks = checkpoint_fine)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
256/256 [==============================] - 1303s 5s/step - loss: 4390.3710 - accuracy: 0.8199
Epoch 2/5
256/256 [==============================] - 1333s 5s/step - loss: 4396.4240 - accuracy: 0.8187
Epoch 3/5
256/256 [==============================] - 1327s 5s/step - loss: 4402.7366 - accuracy: 0.8191
Epoch 4/5
256/256 [==============================] - 1302s 5s/step - loss: 4415.7594 - accuracy: 0.8197
Epoch 5/5
256/256 [==============================] - 1326s 5s/step - loss: 4419.4936 - accuracy: 0.8195


In [ ]:
for img in os.listdir("/content/drive/MyDrive/ResNet/data/proc_img/proc_img_dir/"):
  pic = cv2.imread("/content/drive/MyDrive/ResNet/data/proc_img/proc_img_dir/" + img)
  pic = np.reshape(pic, (1, 224, 224, 3))
  infer = model.predict(pic)
  infer = np.reshape(infer, (64, 64, 3))
  infer[infer < 0.5] = 0
  infer[:, :, 0] *= 255
  infer[:, :, 1] *= 255
  infer[:, :, 2] *= 255
  cv2.imwrite("/content/drive/MyDrive/ResNet/outputs/" + img, infer)